# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 9:09AM,252950,10,0086242895,ISDIN Corporation,Released
1,Dec 13 2022 9:09AM,252950,10,0086242902,ISDIN Corporation,Released
2,Dec 13 2022 9:09AM,252950,10,0086242913,ISDIN Corporation,Released
3,Dec 13 2022 9:09AM,252950,10,0086242920,ISDIN Corporation,Released
4,Dec 13 2022 9:09AM,252950,10,0086242921,ISDIN Corporation,Released
5,Dec 13 2022 9:09AM,252950,10,0086242943,ISDIN Corporation,Released
6,Dec 13 2022 9:09AM,252950,10,0086242975,ISDIN Corporation,Released
7,Dec 13 2022 9:09AM,252950,10,0086245836,ISDIN Corporation,Released
8,Dec 13 2022 9:03AM,252949,10,8893761,"Senseonics, Incorporated",Released
9,Dec 13 2022 9:02AM,252948,10,8892583,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,252946,Released,1
33,252947,Released,1
34,252948,Released,6
35,252949,Released,1
36,252950,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252946,NaN,NaN,1.0
252947,NaN,NaN,1.0
252948,NaN,NaN,6.0
252949,NaN,NaN,1.0
252950,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252812,6.0,0.0,12.0
252813,0.0,0.0,1.0
252818,0.0,1.0,2.0
252823,0.0,1.0,0.0
252832,0.0,1.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252812,6,0,12
252813,0,0,1
252818,0,1,2
252823,0,1,0
252832,0,1,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252812,6,0,12
1,252813,0,0,1
2,252818,0,1,2
3,252823,0,1,0
4,252832,0,1,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252812,6,,12
1,252813,,,1
2,252818,,1,2
3,252823,,1,
4,252832,,1,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation
8,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated"
9,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC"
15,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC"
16,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc"
17,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc."
18,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc."
23,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd"
24,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc."
25,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation,,,8
1,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated",,,1
2,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC",,,6
3,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",,,1
4,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",,,1
5,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",,,1
6,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",,,5
7,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd",,1,
8,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc.",,,1
9,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC",,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation,8,,
1,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated",1,,
2,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC",6,,
3,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1,,
4,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1,,
5,Dec 13 2022 8:36AM,252943,19,"GCH Granules USA, Inc.",1,,
6,Dec 13 2022 8:34AM,252942,20,"Exact-Rx, Inc.",5,,
7,Dec 13 2022 7:26AM,252937,19,"AdvaGen Pharma, Ltd",,1,
8,Dec 12 2022 7:31PM,252936,21,"NBTY Global, Inc.",1,,
9,Dec 12 2022 5:46PM,252934,15,"Virtus Pharmaceuticals, LLC",20,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation,8,,
1,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated",1,,
2,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC",6,,
3,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1,,
4,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation,8.0,NaN,NaN
1,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC",6.0,NaN,NaN
3,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 9:09AM,252950,10,ISDIN Corporation,8.0,0.0,0.0
1,Dec 13 2022 9:03AM,252949,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Dec 13 2022 9:02AM,252948,10,"Snap Medical Industries, LLC",6.0,0.0,0.0
3,Dec 13 2022 8:56AM,252947,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Dec 13 2022 8:54AM,252946,12,"Uniderm USA, Inc",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3540448,84.0,3.0,6.0
12,505874,2.0,0.0,0.0
15,758799,163.0,0.0,0.0
16,505846,2.0,0.0,0.0
19,1517359,7.0,15.0,0.0
20,505804,6.0,22.0,0.0
21,505759,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3540448,84.0,3.0,6.0
1,12,505874,2.0,0.0,0.0
2,15,758799,163.0,0.0,0.0
3,16,505846,2.0,0.0,0.0
4,19,1517359,7.0,15.0,0.0
5,20,505804,6.0,22.0,0.0
6,21,505759,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,84.0,3.0,6.0
1,12,2.0,0.0,0.0
2,15,163.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,7.0,15.0,0.0
5,20,6.0,22.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,2.0
2,15,Released,163.0
3,16,Released,2.0
4,19,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,0.0,0.0,15.0,22.0,1.0
Released,84.0,2.0,163.0,2.0,7.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,15.0,22.0,1.0
2,Released,84.0,2.0,163.0,2.0,7.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,0.0,0.0,15.0,22.0,1.0
2,Released,84.0,2.0,163.0,2.0,7.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()